Step 1: download the data from eotdl

In [ ]:
import os
import zipfile
import eotdl
from eotdl.datasets import download_dataset

download_dataset("EuroSAT", version=1, path="data", force=True)

os.makedirs("data/EuroSAT", exist_ok=True)

with zipfile.ZipFile("data/EuroSAT/v1/EuroSAT.zip", 'r') as zip_ref:
    zip_ref.extractall("data/EuroSAT")

Step 2: Make a random selection of labelled data for which we want to extract EO data, from the images we can readily extract their geospatial bounding box. We append this information into a dataframe which will lateron be used to orchestrate the EO data extraction.

In [2]:
from dataframe_utils import *

# Constants
src_dir = r"C:\Git_projects\eotdl_wip\data\EuroSAT\ds\images\remote_sensing"
num_files = 100 #number of images we wish to process

start_date = "2020-01-01"
nb_months = 3

distance_m = 320  # Buffer distance in meters
resolution = 20.0  # Grid resolution in meters

tif_files = get_tif_files(src_dir)
selected_files = random.sample(tif_files, num_files)

base_df = generate_geodataframe_pet_utm(selected_files, start_date, nb_months, distance_m, resolution)
split_df = process_split_jobs(base_df, 10)
job_df = create_job_dataframe_s2(split_df)


c:\Users\VROMPAYH\AppData\Local\anaconda3\envs\eotdl_openeo_env\lib\site-packages\openeo_gfmap\manager\job_splitters.py:80: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  polygons["centroid"] = polygons.geometry.centroid
c:\Users\VROMPAYH\AppData\Local\anaconda3\envs\eotdl_openeo_env\lib\site-packages\openeo_gfmap\manager\job_splitters.py:84: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  s2_grid["geometry"] = s2_grid.geometry.centroid
c:\Users\VROMPAYH\AppData\Local\anaconda3\envs\eotdl_openeo_env\lib\site-packages\geopandas\array.py:403: UserWarning: Geometry is in a geographic CRS. Results from 'sjoin_nearest' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before t

Subselection for quick testing

In [1]:
# TODO; remove: for testing
job_df = job_df[job_df['feature_count'] > 1][0:1]
job_df

NameError: name 'job_df' is not defined

Step 3: set up a job manager

job_df

In [61]:
import openeo
from openeo.extra.job_management import MultiBackendJobManager, CsvJobDatabase

# Authenticate and add the backend
connection = openeo.connect(url="openeo.dataspace.copernicus.eu").authenticate_oidc()

# initialize the job manager
manager = MultiBackendJobManager()
manager.add_backend("cdse", connection=connection, parallel_jobs=2)

job_tracker = 'job_tracker.csv'
job_db = CsvJobDatabase(path=job_tracker)
if not job_db.exists():
    df = manager._normalize_df(job_df)
    job_db.persist(df)

    

Authenticated using refresh token.


fire up the manager

Code when moving towards UDP based manager

In [ ]:
import openeo
from openeo.extra.job_management import (
        create_job_db,
        ProcessBasedJobCreator,
    )

job_db = create_job_db("jobs.csv", job_df)

process_graph_url = "https://github.com/earthpulse/eotdl/blob/69d17e834d71c05253f3b4d8c8127a3c166192be/tutorials/notebooks/openeo/s1_weekly_statistics.json"

#Make use of the Best Available Pixel openeo Process to obtain Sentinel 2 composites
job_starter = ProcessBasedJobCreator(
        namespace=process_graph_url,
        parameter_defaults={},
    )


In [ ]:
from openeo.extra.job_management import MultiBackendJobManager

# Initiate MultiBackendJobManager 
job_manager = MultiBackendJobManager()  
connection = openeo.connect(url="openeo.dataspace.copernicus.eu").authenticate_oidc()
job_manager.add_backend("cdse", connection=connection, parallel_jobs=10)


job_manager.run_jobs(job_db=job_db, start_job=job_starter)


In [ ]:
import xarray as xr
import matplotlib.pyplot as plt

# Load the dataset
test = xr.open_dataset('test.nc')

# Select a specific time slice for plotting, e.g., the first time point
time_index = 1
b04 = test['B04'].isel(t=time_index)

# Create a figure to plot 4 subplots
fig, axs = plt.subplots(2, 2, figsize=(12, 10))

# Plot each variable in a different subplot
im1 = axs[0, 0].imshow(b04, cmap='viridis', origin='lower')
axs[0, 0].set_title('B04_P10')
fig.colorbar(im1, ax=axs[0, 0])




TODO:


- Scale up (saving data, improve efficiency, ...)

